## Grundlagen GeoPandas

**GeoPandas**
- erbt Funktionen von Pandas, damit ist eine sehr mächtige Tabellenkalkulation möglich
- kann sehr viele Datenformate Lesen und Schreiben, da es auf Fiona und somit auf GDAL aufbaut
- verfügt die wichtigsten Overlay-Funktionen wie **intersection**, **union**, **difference** und andere
- verfügt über die wichtigsten geometrischen Manipulationsmethoden wie **buffer**, **centroid**, **convex_hull** und weitere
- kennt die wichtigsten affinen Transformationen wie **rotate**, **scale**, **transform**, **affine_transform (matrix)**
- ist sehr schnell bei Berechnungen und Umwandlungen, da diese nicht zeilenbasiert sondern matrixbasiert durchgeführt werden, insbesondere Laden, Joinen und gruppieren von Daten.

### Datenformate

In [ ]:
# Unterstützte Datenformate:
import fiona 
fiona.supported_drivers

Es können auch webbasierte Datenformate wie *geojson* direkt über eine URL geladen werden:

In [ ]:
import geopandas as gpd
url = "http://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_land.geojson"
df = gpd.read_file(url)
df.plot()

### Datenfilter
Da GeoPandas auf Fiona aufbaut, was wiederum auf GDAL basiert, lassen sich Daten bereits beim Einlesen filtern:

In [ ]:
# Geometrischer Filter:
gdf_mask = gpd.read_file("data/countries.shp")

#Alle Städte in Afrika einlesen
gdf = gpd.read_file("data/cities.shp",
    mask=gdf_mask[gdf_mask.continent=="Africa"],
)
gdf.plot()

oder mit einer Bounding Box:

In [ ]:
bbox = (
    -100, -50, -30, 20
)
gdf = gpd.read_file("data/cities.shp",
    bbox=bbox,
)
gdf.plot()

### Geschwindigkeit:
Im Shapefile 'buildings_a.shp' - mit 41.000 Zeilen - eine neue Spalte [area] anlegen und die Gebäudeflächen berechnen. Mit **QGIS** hat mein Rechner dafür knapp 3,5 Sekunden mit dem Feldrechner benötigt, in **ArcMap** ca. 5 Sekunden.  

Mit einen PYQGIS Script der Art:  

`with edit(layer):
    for feature in layer.getFeatures():
        feature['area'] = feature.geometry().area()
        layer.updateFeature(feature)`  
        
dauerte es sogar noch deutlich länger: 18 Minuten!

In [1]:
import geopandas as gpd
import datetime #Performance messen
#Startzeitder Berechnung für Performancemessung
t1 = datetime.datetime.now()
#Daten einlesen
fp = "data/buildings_a.shp"
df = gpd.read_file(fp)

#Zwischenzeit nehmen
t2 = datetime.datetime.now()
secs = (t2-t1).total_seconds()
print("Reading data in {} seconds".format(secs))
t1 = datetime.datetime.now()

#Neue Spalte 'area' mit Gebäudefläche bestimmen:
df["area"] = df['geometry'].area

#Zeit berechnen für Flächenberechnung
t2 = datetime.datetime.now()
secs = (t2-t1).total_seconds()
print("Area-calculation done in {} seconds".format(secs))
df.head()

Reading data in 1.502301 seconds
Area-calculation done in 0.062481 seconds


,fid,osm_id,code,fclass,name,type,geometry,area
0,1.0,4525597,1500,building,Sankt Thomas Kirche,church,"POLYGON ((869156.302 6108442.913, 869173.868 6...",549.460723
1,2.0,17062362,1500,building,Konzerthaus Freiburg,None,"POLYGON ((872938.827 6106136.537, 872939.806 6...",10937.636339
2,3.0,17062367,1500,building,None,civic,"POLYGON ((873818.763 6107470.788, 873895.807 6...",10667.333809
3,4.0,17062368,1500,building,Novotel,hotel,"POLYGON ((872879.972 6106062.495, 872913.702 6...",3483.000274
4,5.0,17062369,1500,building,Intercity Hotel,hotel,"POLYGON ((872897.071 6106396.646, 872907.969 6...",1417.244249


Mit der `describe()` Funktion können wir uns nun die Statistik zu den Daten sehr schnell ansehen:

In [ ]:
df.describe()